In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
import datetime




In [ ]:
# Load the data
data = np.load('flatland_train.npz')
X = data['X']
y = data['y']


In [ ]:
# Preprocess the data
X = X / 255.0
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)


The pixel values of the images are normalized to the range [0, 1] by dividing by 255.0. The labels (y) are encoded using LabelEncoder and then converted to one-hot encoded format using to_categorical.

In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



The data is split into training and testing sets using the train_test_split function from scikit-learn. 80% of the data is used for training (X_train, y_train), and 20% is used for testing (X_test, y_test).

In [ ]:
# Ensure the input data has the correct shape (add channel dimension for grayscale images)
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


The ImageDataGenerator is configured with various transformations like rotation, width shift, height shift, shear, zoom, horizontal flip, and vertical flip. This generator will be used to augment the training data during model training.

In [ ]:
# Build the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))



The model is built using the Sequential API from Keras. It consists of convolutional layers (Conv2D), batch normalization layers (BatchNormalization), max-pooling layers (MaxPooling2D), a flattening layer (Flatten), dense layers (Dense), and a dropout layer (Dropout). The final layer has softmax activation, suitable for multiclass classification.

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


The model is compiled with the Adam optimizer, categorical crossentropy loss (suitable for multiclass classification), and accuracy as the evaluation metric.

In [ ]:
# Model callbacks
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model with data augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,
                    epochs=50,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, early_stopping, tensorboard])


Epoch 1/50
250/250 [==============================] - ETA: 0s - loss: 1.7681 - accuracy: 0.3666
Epoch 1: val_loss improved from inf to 1.97891, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 42s 161ms/step - loss: 1.7681 - accuracy: 0.3666 - val_loss: 1.9789 - val_accuracy: 0.2195
Epoch 2/50
250/250 [==============================] - ETA: 0s - loss: 1.3619 - accuracy: 0.4518
Epoch 2: val_loss improved from 1.97891 to 1.64817, saving model to best_model.h5
250/250 [==============================] - 39s 156ms/step - loss: 1.3619 - accuracy: 0.4518 - val_loss: 1.6482 - val_accuracy: 0.1850
Epoch 3/50
250/250 [==============================] - ETA: 0s - loss: 0.9887 - accuracy: 0.5824
Epoch 3: val_loss improved from 1.64817 to 0.66472, saving model to best_model.h5
250/250 [==============================] - 40s 160ms/step - loss: 0.9887 - accuracy: 0.5824 - val_loss: 0.6647 - val_accuracy: 0.7585
Epoch 4/50
250/250 [==============================] - ETA: 0s - loss: 0.7470 - accuracy: 0.7091
Epoch 4: val_loss did not improve from 0.66472
250/250 [==============================] - 39s 155ms/step - loss: 0.7470 - accuracy: 0.7091 - val_l

The model is trained using the fit method. The training data is augmented using the datagen.flow generator. The training process is monitored by callbacks, including model checkpointing (ModelCheckpoint), early stopping (EarlyStopping), and TensorBoard logging (TensorBoard).

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

63/63 [==============================] - 2s 29ms/step - loss: 0.1518 - accuracy: 0.9820
Test Accuracy: 0.9819999933242798


The trained model is evaluated on the test set using the evaluate method. The test accuracy is then printed.

In [ ]:
# Save the final model
model.save('model.h5')